In [10]:
import openai
import pymupdf4llm
import pymupdf
import json
from fuzzysearch import find_near_matches
from langchain.text_splitter import MarkdownTextSplitter
import pandas as pd
import openai
import json
import pymupdf4llm
import pymupdf

from comments import find_comments_in_doc, get_historic_sites_comments, get_species_comments

from tqdm import tqdm

In [11]:
PAGES_PER_CHUNK = 50

In [12]:
# Load EIS document
page_count = pymupdf.open("sample.pdf").page_count
full_doc = pymupdf4llm.to_markdown("sample.pdf")
doc_chunks = [
    pymupdf4llm.to_markdown("sample.pdf", pages=range(i, i + PAGES_PER_CHUNK))
    for i in range(0, page_count, PAGES_PER_CHUNK)
]

In [13]:
# Load context
sites = pd.read_excel("sources/sites.xlsx")
species = pd.read_csv("sources/species.csv")

In [14]:
final_comments = []
for chunk in tqdm(doc_chunks):
    site_comments = get_historic_sites_comments(chunk, sites)
    species_comments = get_species_comments(chunk, species)
    final_comments.extend(site_comments['comments'] + species_comments['comments'])

100%|██████████| 6/6 [02:53<00:00, 28.91s/it]


In [15]:
len(final_comments)

28

In [16]:
found_comments = find_comments_in_doc(full_doc, final_comments)

In [17]:
len(found_comments)

18

In [19]:
output = {"markdown": full_doc, "comments": found_comments}
with open("final_1.json", "w") as f:
    f.write(json.dumps(output, indent=4))